# KG Final Coursework Notebook

Many references taken from the lab sessions (4, 5, 6, 7, 8 ,9)

Much of the code has changed and done in blocks to fit the different data

empty descriptions could have been parsed from the pizza name for example mushroom pizza has ingredient mushroom but this was left out

## Installing libraries

In [1]:
#Jupyter notebook
#jupyter==1.0.0
#!pip install SPARQLWrapper==1.8.5
#!pip install Owlready2==0.36
#!pip install requests==2.25.1
#!pip install pyparsing==2.4.7
#!pip install rdflib==5.0.0
#!pip install pandas==2.2.2
#!pip install python-Levenshtein==0.12.2 #had trouble installing this "could not build wheels"
#!pip install owlrl==5.2.1
#!pip install lookup
#!pip install isub
#!pip install cydifflib==1.1.0 #alternative for distance
#!pip install gensim==4.3.1

## Importing Libraries

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from isub import isub
# Look-ups
from lookup import WikidataAPI, GoogleKGLookup
from endpoints import WikidataEndpoint
import pandas as pd
import sys
import csv
import owlrl
from owlready2 import *
#Shoudl be imported after owlready
import string
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from rdflib.util import guess_format
import re
import difflib

C:\Users\Fasih Munir\anaconda3\envs\knowledge_graphs\Lib\site-packages\rdflib_jsonld\__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  warnings.warn(
* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## 2.2 Tabular Data to Knowledge Graph (Task RDF)

### Subtask RDF.0

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
#lets load our data into a dataframe and look around and see what we have

cw_data = pd.read_csv('cw_data.csv')
cw_data.head(5)

#as expected, looks like we have some pizzas to work with

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description
0,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701.0,OR,Pizza Place,Bianca Pizza,22.50,USD,NaN
1,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701.0,OR,Pizza Place,Cheese Pizza,18.95,USD,NaN
2,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Margherita",12.00,USD,NaN
3,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Mushroom",13.00,USD,NaN
4,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Puttenesca",13.00,USD,"Olives, onions, capers, tomatoes"


In [5]:
#working with a copy

df_cw = cw_data.copy()

In [6]:
#lets add a few more pizza examples to the end of the data frame
#we will add 2 more fake examples
#using information from the onotology

more_pizza = {
    'name': ['Foolish Fasih Pizza', 'Fabulous Fasih Pizza', 'Foolish Fasih Pizza', 'Fabulous Fasih Pizza'], #made up
    'address': ['123 Foolish Street', '123 Fabulous Street', '123 Foolish Street', '123 Fabulous Street'], #made up
    'city': ['Los Angeles', 'Los Angeles', 'Los Angeles', 'Los Angeles'],
    'country': ['US', 'US', 'US', 'US'],
    'postcode': [90049.0, 90049.0, 90049.0, 90049.0],
    'state': ['Brentwood', 'Brentwood', 'Brentwood', 'Brentwood'],
    'categories': ['Pizza Place', 'Pizza Place', 'Pizza Place', 'Pizza Place'],
    'menu item': ['Fools Gold', 'Mighty Chicken', 'Fickle Feast', 'Glamorous Greens'],
    'item value': [5.00, 5.00, 3.00, 30.00],
    'currency': ['USD', 'USD', 'USD', 'USD'],
    'item description': ['Chicken, Yellow pepper', 'Chicken', 'Ham, Capers', 'Beans, Onion, Mushroom']
}

more_pizza_df = pd.DataFrame(more_pizza)
df_cw = pd.concat([df_cw, more_pizza_df], ignore_index=True)
df_cw.tail(4)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description
501,Foolish Fasih Pizza,123 Foolish Street,Los Angeles,US,90049.0,Brentwood,Pizza Place,Fools Gold,5.0,USD,"Chicken, Yellow pepper"
502,Fabulous Fasih Pizza,123 Fabulous Street,Los Angeles,US,90049.0,Brentwood,Pizza Place,Mighty Chicken,5.0,USD,Chicken
503,Foolish Fasih Pizza,123 Foolish Street,Los Angeles,US,90049.0,Brentwood,Pizza Place,Fickle Feast,3.0,USD,"Ham, Capers"
504,Fabulous Fasih Pizza,123 Fabulous Street,Los Angeles,US,90049.0,Brentwood,Pizza Place,Glamorous Greens,30.0,USD,"Beans, Onion, Mushroom"


### Subtask RDF.1 and RDF.2

uri generation

NOTE- american restaurant query wont work since its not been passed as type. this is because it was a subclass not child class. i have only shown the absolute child classes here. in reasoning a mexican restuarnt would be classified as an american restaurant else there are not american restaurants. i followed the logic in the ontology

In [7]:
#creating graph and loading onotology

g = Graph()

#loading onotology to get a list of classes
g.parse('pizza-restaurants-ontology.ttl', format='ttl')

#setting up the namespace and uri convention as well
cw_namespace = 'http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#'
cw = Namespace(cw_namespace)
g.bind('cw', cw)

rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
g.bind("rdf", rdf)
g.bind("xsd", xsd)

In [8]:
#the number of triples before any uri generation

print(str(len(g)))

963


In [9]:
#simple query to get distinct class list. had to parse a lot of documentation and use my sql brain
#documentation: https://codyburleson.com/blog/sparql-examples-list-classes
#documentation: https://www.w3.org/TR/rdf-sparql-query/#sparqlQueryVariables

distinct_class_query = """
SELECT DISTINCT ?class
WHERE {
    ?class a owl:Class .
}
"""

#running the query
distinct_classes = g.query(distinct_class_query)

class_names = [classes[0].split('#')[-1] for classes in distinct_classes]
class_names

#i used this list the first time and then realized it was making multiple tags for things that it shouldnt example in menu item it would look for pepper
#and pepper is not a pizza its just part of pepporoni
#and thats wrong
#so then getting sublists and passing everything through there

['Oregano',
 'Club',
 'ChickenPizza',
 'Pepper',
 'Country',
 'Shrimp',
 'City',
 'Scallops',
 'RootVegetable',
 'GourmetRestaurants',
 'Ingredient',
 'BarAndGrill',
 'Pepperoni',
 'Meatballs',
 'AmericanPizza',
 'Beef',
 'BurgerPlace',
 'Chicken',
 'PizzaByStyle',
 'ub1bL1387C40',
 'ub1bL1015C38',
 'Pub',
 'BeansPizza',
 'MenuItem',
 'Ricotta',
 'SportsBar',
 'SushiRestaurant',
 'ub1bL963C79',
 'HotSauce',
 'KaraokeBar',
 'GreekRestaurant',
 'Salmon',
 'ub1bL580C58',
 'ub1bL1001C80',
 'MargheritaPizza',
 'Olives',
 'ub1bL1030C81',
 'GreenOlives',
 'Seeds',
 'Pineapple',
 'Pesto',
 'GreenPepper',
 'Tomato',
 'Mozzarella',
 'Address',
 'DietaryRestaurant',
 'Herbs',
 'State',
 'PizzaSupreme',
 'JapanesePizza',
 'BlackOlives',
 'ub1bL1061C83',
 'PizzaMarinara',
 'CherryTomato',
 'MexicanPizza',
 'Marinara',
 'Carrot',
 'ub1bL820C37',
 'AmericanRestaurant',
 'BeerPlace',
 'ub1bL771C34',
 'IndianRestaurant',
 'Gorgonzola',
 'Vegetable',
 'Pizzeria',
 'Spinach',
 'VegetarianPizza',
 'Anchov

In [10]:
#list for menu items

distinct_menu_item_query = """
SELECT DISTINCT ?subclass
WHERE {
  ?subclass rdfs:subClassOf* <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Pizza> .
  FILTER (?subclass != <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Pizza>)
}
"""

menu_items = g.query(distinct_menu_item_query)

menu_item_names = [items[0].split('#')[-1] for items in menu_items]

#manually removing additional sub classes but could not figure out how to do this in the query
excluded_menu_items = {"NamedPizza", "PizzaByIngredient", "MargheritaPizza", "PizzaByStyle", 
                  "AmericanPizza", "SweetPizza", "VegetarianPizza"}

#filtering with list comprehension
filtered_menu_item_names = [menu_item for menu_item in menu_item_names if menu_item not in excluded_menu_items]

#doing this in the loop itself
#removing the word pizza as pizza_bianca is not the same as bianca_pizza
#all items have pizza in them and are unique without the pizza as well so no problem
#but we will end up with things like if the name of the pizza includes the name of the restaurant and the restaurant name is like greek_city
#and they sell greek_city_chicken_pizza - that can be tagged as a greek pizza and a chicken pizza
#cleaned_menu_item_names = [name.replace("Pizza", "") for name in filtered_menu_item_names]

filtered_menu_item_names
#cleaned_menu_item_names

['CalifornianPizza',
 'MexicanPizza',
 'HawaiianPizza',
 'MeatPizza',
 'JapanesePizza',
 'SeafoodPizza',
 'PizzaNutella',
 'PizzaRomana',
 'GreekPizza',
 'FruitPizza',
 'VeganPizza',
 'PizzaBianca',
 'PizzaSupreme',
 'PizzaErnesto',
 'PizzaNapolitana',
 'PizzaMarinara',
 'FetaPizza',
 'PineapplePizza',
 'BarbecuePizza',
 'MushroomPizza',
 'BeansPizza',
 'ChickenPizza']

In [11]:
#list for ingredients
#this will miss out on things like tomato
#tomato is a subclass that holds cherry tomato
#but not tomato itself
#so if an ingredient is listed as tomoto, that uri wont be generated
#i could add another loop into the code later down but i was not able to figure out how to stop it from creating multiple uris so i left this for now
#i could also just add tomato to the list but that not being able to do it for the others i left it out

distinct_ingredients_query = """
SELECT DISTINCT ?subclass
WHERE {
  ?subclass rdfs:subClassOf* <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Ingredient> .
  FILTER (?subclass != <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Ingredient>)
}
"""

ingredients = g.query(distinct_ingredients_query)

ingredients_names = [ingredient[0].split('#')[-1] for ingredient in ingredients]

#ingredients_names

#manually removing additional sub classes but could not figure out how to do this in the query
excluded_ingredients = {"Meat", "Sausage", "Sauce", "TomatoSauce", 
                        "SeaFood", "VegetarianIngredient", "Cheese",
                        "VeganIngredient", "Vegetable", "Fruit",
                        "Herbs", "Olives", "Pepper",
                        "RootVegetable", "Seeds", "Tomato",
                        "WinterSquash"}

#filtering with list comprehension
filtered_ingredients_names = [ingredient for ingredient in ingredients_names if ingredient not in excluded_ingredients]

filtered_ingredients_names

['Chorizo',
 'Mortadella',
 'Prosciutto',
 'Bacon',
 'Ham',
 'Salami',
 'Meatballs',
 'Chicken',
 'Beef',
 'Pepperoni',
 'Anchovies',
 'Salmon',
 'Scallops',
 'Tuna',
 'Shrimp',
 'CrabMeat',
 'Marinara',
 'Pesto',
 'HotSauce',
 'BarbecueSauce',
 'OliveOil',
 'Rosemary',
 'Oregano',
 'Basil',
 'JalapenoPepper',
 'GreenPepper',
 'RedPepper',
 'YellowPepper',
 'Eggplant',
 'Broccoli',
 'Spinach',
 'PlumTomato',
 'CherryTomato',
 'Onion',
 'Capers',
 'Potato',
 'Carrot',
 'SweetPotato',
 'Artichokes',
 'Mushroom',
 'Pumpkin',
 'ButternutSquash',
 'Garlic',
 'Beans',
 'Fig',
 'Pineapple',
 'Zucchini',
 'BlackOlives',
 'GreenOlives',
 'Tofu',
 'VeganCheese',
 'Gorgonzola',
 'Mozzarella',
 'Parmesan',
 'GoatCheese',
 'Feta',
 'Provolone',
 'Ricotta',
 'Cheddar',
 'BlueCheese']

In [12]:
#list for categories/restaurants
#NOTE- american restaurant query wont work since its not been passed as type
#this is because it was a subclass not child class
#i have only shown the absolute child classes here
#in reasoning a mexican restuarnt would be classified as an american restaurant else there are not american restaurants
#i followed the logic in the ontology

distinct_restaurant_types_query = """
SELECT DISTINCT ?subclass
WHERE {
  ?subclass rdfs:subClassOf* <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Restaurant> .
  FILTER (?subclass != <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Restaurant>)
}
"""

restaurant_types = g.query(distinct_restaurant_types_query)

restaurant_types_names = [restaurant_type[0].split('#')[-1] for restaurant_type in restaurant_types]

#manually removing additional sub classes but could not figure out how to do this in the query
excluded_restaurant_types = {"AmericanRestaurant", "AsianRestaurant", "JapaneseRestaurant", "BarsPubsClubs", 
                        "Bar", "Pub", "DietaryRestaurant",
                        "MediterraneanRestaurant"}

#filtering with list comprehension
filtered_restaurant_types_names = [restaurant_type for restaurant_type in restaurant_types_names if restaurant_type not in excluded_restaurant_types]

filtered_restaurant_types_names

['CoffeeShop',
 'SpanishRestaurant',
 'FrenchRestaurant',
 'ItalianRestaurant',
 'GreekRestaurant',
 'Gastropub',
 'BeerPlace',
 'CocktailBars',
 'SportsBar',
 'KaraokeBar',
 'Club',
 'SeafoodRestaurant',
 'BurgerPlace',
 'ChineseRestaurant',
 'SushiRestaurant',
 'IndianRestaurant',
 'PizzaPlace',
 'BarAndGrill',
 'VegetarianRestaurant',
 'GlutenFreeRestaurant',
 'VeganRestaurant',
 'GourmetRestaurants',
 'Bakery',
 'Pizzeria',
 'MexicanRestaurant']

In [13]:
#now lets add 2 columns to get some uniqueness for the restaurant names and the menu items

df_cw['name_city'] = df_cw['name'] + '_' + df_cw['city']
df_cw['name_menu_item'] = df_cw['name'] + '_' + df_cw['menu item']
df_cw.head(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item
0,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701.0,OR,Pizza Place,Bianca Pizza,22.50,USD,NaN,Little Pizza Paradise_Bend,Little Pizza Paradise_Bianca Pizza
1,Little Pizza Paradise,Cascade Village Mall Across From Target,Bend,US,97701.0,OR,Pizza Place,Cheese Pizza,18.95,USD,NaN,Little Pizza Paradise_Bend,Little Pizza Paradise_Cheese Pizza
2,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Margherita",12.00,USD,NaN,The Brentwood_Los Angeles,"The Brentwood_Pizza, Margherita"
3,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Mushroom",13.00,USD,NaN,The Brentwood_Los Angeles,"The Brentwood_Pizza, Mushroom"
4,The Brentwood,148 S Barrington Ave,Los Angeles,US,90049.0,Brentwood,"American Restaurant,Bar,Bakery","Pizza, Puttenesca",13.00,USD,"Olives, onions, capers, tomatoes",The Brentwood_Los Angeles,"The Brentwood_Pizza, Puttenesca"


In [14]:
#now lets make a little pre processing function
#the data has some bits of apostrophes, spaces, commas and uppercasing
#to standardize everything these would need to be dealt with
#looking at the data there are 2 issues that would while standardizing, the apostrophes and the underscores doubling up
#so we will first remove apostrophe, then all punctuation and space gets changed to underscore
#then more than 1 underscore is changed to single
#finally we lower case

def text_preprocessing(column):
    
    #remove aspostrophes
    column = column.str.replace("'", "")

    #translation table for punctation
    #documentation: https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
    translator = str.maketrans(string.punctuation, '_' * len(string.punctuation))
    column = column.str.translate(translator)

    #replace only spacse
    #documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
    column = column.str.replace(' ', '_', regex=True)

    #replacing double underscores
    #documentation: https://docs.python.org/3/library/re.html
    column = column.str.replace(r'__+', '_', regex=True)

    #additional check just to make it easy during uri creation
    column = column.str.strip('_')
    
    #lowercasing
    column = column.str.lower()

    return column

In [15]:
df_cw['name_city'] = text_preprocessing(df_cw['name_city'])
df_cw['name_menu_item'] = text_preprocessing(df_cw['name_menu_item'])
df_cw['address'] = text_preprocessing(df_cw['address'])
df_cw['city'] = text_preprocessing(df_cw['city'])
df_cw['country'] = text_preprocessing(df_cw['country'])
df_cw['state'] = text_preprocessing(df_cw['state'])
df_cw['categories'] = text_preprocessing(df_cw['categories'])
df_cw['currency'] = text_preprocessing(df_cw['currency'])
df_cw['item description'] = text_preprocessing(df_cw['item description'])

In [16]:
df_cw.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item
500,John's of 12th Street,302_e_12th_st,new_york,us,10003.0,manhattan,restaurant_italian_restaurant_italian_restaurant_east_village,Vegan Margherita Pizza,16.95,usd,tomato_basil_garlic_daiya_cheese,johns_of_12th_street_new_york,johns_of_12th_street_vegan_margherita_pizza
501,Foolish Fasih Pizza,123_foolish_street,los_angeles,us,90049.0,brentwood,pizza_place,Fools Gold,5.00,usd,chicken_yellow_pepper,foolish_fasih_pizza_los_angeles,foolish_fasih_pizza_fools_gold
502,Fabulous Fasih Pizza,123_fabulous_street,los_angeles,us,90049.0,brentwood,pizza_place,Mighty Chicken,5.00,usd,chicken,fabulous_fasih_pizza_los_angeles,fabulous_fasih_pizza_mighty_chicken
503,Foolish Fasih Pizza,123_foolish_street,los_angeles,us,90049.0,brentwood,pizza_place,Fickle Feast,3.00,usd,ham_capers,foolish_fasih_pizza_los_angeles,foolish_fasih_pizza_fickle_feast
504,Fabulous Fasih Pizza,123_fabulous_street,los_angeles,us,90049.0,brentwood,pizza_place,Glamorous Greens,30.00,usd,beans_onion_mushroom,fabulous_fasih_pizza_los_angeles,fabulous_fasih_pizza_glamorous_greens


In [17]:
#creating a function to transform class names
#initally i made a new list but was not able to get this to work
#documentation: https://stackoverflow.com/questions/7322028/how-to-replace-uppercase-with-underscore
#documentation: https://docs.python.org/3/library/re.html

#our list has things like Eggplant but also SweetPotato so need to account for both
#Eggplant should become eggplant while SweetPotato should become sweet_potato
#done using some simple logic and regex
#essentially using ?<= and ?= we are searching for the positions where lower case becomes upper case
#this is a common snake_case transformation
def transform_class_name(class_name):
    if re.search(r'(?<=[a-z])(?=[A-Z])', class_name):
        transformed = re.sub('(?<=[a-z])(?=[A-Z])', '_', class_name).lower()
    else:
        transformed = class_name.lower()
    return transformed

#### URIs and Triples for Classes

##### URIs for Item Description

first i tried to preprocess the data and get every ingredient as its own row using split and explode but this was proving difficult because the preproessing was resulting in more outlier cases

then i decided to pull a list from the ontology and loop through the list by matching sub strings which seemed to do most of the trick

In [18]:
#getting uris for item description

#for loop to update the graph
#first i was matching exact strings then loosened the restriction to check for anywhere in the string
#i was also referencing a previously made list which was not giving me all the results so fixed that small bug
#essentially for every class name in our list class names, we transform the class and save that in a variable
#then for every row in our dataframe we check if its na and if the transformed name is in the column
#if it is we create a uri and add to the graph
#else we do nothing

#creating another column in the dataframe to store the uri for that ingredient for that row
#adding rows to append later for computation time
df_cw['ingredient_uri'] = pd.NA
additional_rows = []

for class_name in filtered_ingredients_names:
    transformed_class_name = transform_class_name(class_name)
    for idx, row in df_cw.iterrows():
        if pd.notna(row['item description']) and transformed_class_name in row['item description']:
            ingredient_uri = URIRef(cw_namespace + transformed_class_name)
            #saving the rows as duplicates in a new column
            #will help when aking the object property as can then simply reference the columns
            new_row = row.copy()
            new_row['ingredient_uri'] = str(ingredient_uri)
            additional_rows.append(new_row)
            g.add((ingredient_uri, RDF.type, cw[class_name]))

if additional_rows:
    df_cw = pd.concat([df_cw, pd.DataFrame(additional_rows)], ignore_index=True)

In [19]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'steinway_pizza_grandma_pizza_round_or_square']
filtered_df

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri
391,Steinway Pizza,3151_steinway_st,astoria,us,11103.0,queens,pizza_place,Grandma Pizza (round or Square),17.0,usd,fresh_mozzarella_garlic_tomato_and_basil,steinway_pizza_astoria,steinway_pizza_grandma_pizza_round_or_square,NaN
654,Steinway Pizza,3151_steinway_st,astoria,us,11103.0,queens,pizza_place,Grandma Pizza (round or Square),17.0,usd,fresh_mozzarella_garlic_tomato_and_basil,steinway_pizza_astoria,steinway_pizza_grandma_pizza_round_or_square,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#basil
803,Steinway Pizza,3151_steinway_st,astoria,us,11103.0,queens,pizza_place,Grandma Pizza (round or Square),17.0,usd,fresh_mozzarella_garlic_tomato_and_basil,steinway_pizza_astoria,steinway_pizza_grandma_pizza_round_or_square,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#garlic
884,Steinway Pizza,3151_steinway_st,astoria,us,11103.0,queens,pizza_place,Grandma Pizza (round or Square),17.0,usd,fresh_mozzarella_garlic_tomato_and_basil,steinway_pizza_astoria,steinway_pizza_grandma_pizza_round_or_square,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#mozzarella


In [20]:
#creating a quick query to see what kind of output we get
#dont think this query is exactly right but will get the job done

test_query = """
SELECT ?ingredient ?type
WHERE {
    ?ingredient rdf:type ?type .
}
"""

test_results = g.query(test_query)

for result in test_results:
    print(f"Ingredient URI: {result.ingredient}, Type: {result.type}")

Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#shrimp, Type: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Shrimp
Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#spinach, Type: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Spinach
Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Club, Type: http://www.w3.org/2002/07/owl#Class
Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#ChickenPizza, Type: http://www.w3.org/2002/07/owl#Class
Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#cheddar, Type: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Cheddar
Ingredient URI: ub1bL1096C33, Type: http://www.w3.org/2002/07/owl#Restriction
Ingredient URI: http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#Pepper, Type: http://www.w3.org/2002/07/owl#Class
Ingredient URI: http://www.semanticweb

In [21]:
results_list = list(test_results)
print(f"Total results: {len(results_list)}")

Total results: 322


In [22]:
#number of triples after item description uri

print(str(len(g)))

1017


In [23]:
len(filtered_ingredients_names)

60

In [24]:
#tried search for pumpkin manually in excel and through query but does not seem like pumpkin is an instance even though there is a class
#i think the uris have captured most things then

test_query2 = """
SELECT (COUNT(DISTINCT ?black_olives) AS ?numberblackolives)
WHERE {
  ?black_olives rdf:type cw:BlackOlives .
  FILTER(CONTAINS(STR(?black_olives), "black_olives"))
}
"""

test_results2 = g.query(test_query2)

for result in test_results2:
    print(f"Total Blackolives: {result.numberblackolives}")

Total Blackolives: 1


##### URIs for Menu Item

using our unique column name_menu_iteary

In [25]:
df_cw['menu_item_uri'] = pd.NA
additional_rows = []

for class_name in filtered_menu_item_names:
    cleaned_class_name = class_name.replace("Pizza", "")
    transformed_class_name = transform_class_name(cleaned_class_name)
    uri_class_name = transform_class_name(class_name)
    for idx, row in df_cw.iterrows():
        if pd.notna(row['name_menu_item']) and transformed_class_name in row['name_menu_item']:
            menu_item_uri = URIRef(cw_namespace + row['name_menu_item'])
            new_row = row.copy()
            new_row['menu_item_uri'] = str(menu_item_uri)
            additional_rows.append(new_row)
            g.add((menu_item_uri, RDF.type, cw[class_name]))

if additional_rows:
    df_cw = pd.concat([df_cw, pd.DataFrame(additional_rows)], ignore_index=True)

In [26]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'the_brentwood_pizza_mushroom']
filtered_df

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri
3,The Brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Mushroom",13.0,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_mushroom,NaN,NaN
1113,The Brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Mushroom",13.0,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_mushroom,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_pizza_mushroom


In [27]:
#number of triples after menu item uri

print(str(len(g)))

1142


##### URIs for Currency


In [28]:
#previous code wont work as currency column just holds usd so need to explicitly define a string

df_cw['currency_uri'] = pd.NA
additional_rows = []

currency_class_name = 'Currency'
for idx, row in df_cw.iterrows():
    if pd.notna(row['currency']) and row['currency'] == 'usd':
        currency_uri = URIRef(cw_namespace + 'usd')
        new_row = row.copy()
        new_row['currency_uri'] = str(currency_uri)
        additional_rows.append(new_row)
        g.add((currency_uri, RDF.type, cw[currency_class_name]))

if additional_rows:
    df_cw = pd.concat([df_cw, pd.DataFrame(additional_rows)], ignore_index=True)

In [29]:
#number of triples after menu item uri

print(str(len(g)))

1143


In [30]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri
8,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN,NaN
512,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,NaN
817,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,NaN
983,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN
989,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN
995,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN
1276,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd
1705,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd
1987,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd
2139,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd


##### URIs for Item Value

handling a bit differently since these are only numbers

we will check if the number is a float and then add that number to the uri as a class of item value

using the same loop structure as before just modifying based on need

In [31]:
df_cw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              2398 non-null   object 
 1   address           2398 non-null   object 
 2   city              2398 non-null   object 
 3   country           2398 non-null   object 
 4   postcode          2364 non-null   float64
 5   state             2398 non-null   object 
 6   categories        2398 non-null   object 
 7   menu item         2398 non-null   object 
 8   item value        2246 non-null   float64
 9   currency          2260 non-null   object 
 10  item description  1673 non-null   object 
 11  name_city         2398 non-null   object 
 12  name_menu_item    2398 non-null   object 
 13  ingredient_uri    1220 non-null   object 
 14  menu_item_uri     585 non-null    object 
 15  currency_uri      1130 non-null   object 
dtypes: float64(2), object(14)
memory usage: 29

In [32]:
#documentation: https://www.w3schools.com/python/ref_func_isinstance.asp

df_cw['item_value_uri'] = pd.NA
additional_rows = []

item_value_class_name = 'ItemValue'

for idx, row in df_cw.iterrows():
    if pd.notna(row['item value']) and isinstance(row['item value'], float):
        item_value_uri = URIRef(cw_namespace + str(row['item value']))
        new_row = row.copy()
        new_row['item_value_uri'] = str(item_value_uri)
        additional_rows.append(new_row)
        g.add((item_value_uri, RDF.type, URIRef(cw_namespace + item_value_class_name)))

if additional_rows:
    df_cw = pd.concat([df_cw, pd.DataFrame(additional_rows)], ignore_index=True)

In [33]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri
8,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN,NaN,NaN
512,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,NaN,NaN
817,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,NaN,NaN
983,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN
989,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN
995,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN
1276,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,NaN
1705,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,NaN
1987,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,NaN
2139,Bravo Pizza Hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,NaN


In [34]:
#number of triples after item value uri
print(str(len(g)))

1273


##### URIs for Address

this is a bit like usd as there only some addresses and no empty values

can technically just make uris for each unique address directly

but we will make the address more unique by concatenating all parts

name, address, city, state, country, post code

In [35]:
df_cw.head(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri
0,Little Pizza Paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Bianca Pizza,22.50,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_bianca_pizza,NaN,NaN,NaN,NaN
1,Little Pizza Paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Cheese Pizza,18.95,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_cheese_pizza,NaN,NaN,NaN,NaN
2,The Brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Margherita",12.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_margherita,NaN,NaN,NaN,NaN
3,The Brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Mushroom",13.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_mushroom,NaN,NaN,NaN,NaN
4,The Brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Puttenesca",13.00,usd,olives_onions_capers_tomatoes,the_brentwood_los_angeles,the_brentwood_pizza_puttenesca,NaN,NaN,NaN,NaN


In [36]:
df_cw['name'] = text_preprocessing(df_cw['name'])

In [37]:
#not adding the post code as there are some nans which are duplicating addresses where you have address_nan and address_900909

df_cw['full_address'] = (df_cw['name'] + '_' + 
                         df_cw['address']  + '_' + 
                         df_cw['city'] + '_' + 
                         df_cw['state'] + '_' + 
                         df_cw['country'] 
                         #+ '_' + 
                         #df_cw['postcode'].astype(str)
                        )

df_cw.head(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address
0,little_pizza_paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Bianca Pizza,22.50,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_bianca_pizza,NaN,NaN,NaN,NaN,little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us
1,little_pizza_paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Cheese Pizza,18.95,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_cheese_pizza,NaN,NaN,NaN,NaN,little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us
2,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Margherita",12.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_margherita,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us
3,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Mushroom",13.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_mushroom,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us
4,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Puttenesca",13.00,usd,olives_onions_capers_tomatoes,the_brentwood_los_angeles,the_brentwood_pizza_puttenesca,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us


In [38]:
unique_addresses = df_cw['full_address'].dropna().unique()
#unique_addresses

In [39]:
df_cw['address_uri'] = pd.NA

#iteratino began taking too long here found a way to do it through apply
#documentation: https://stackoverflow.com/questions/16476924/how-can-i-iterate-over-rows-in-a-pandas-dataframe#:~:text=You%20can%20also%20use%20df,multiple%20columns%20for%20a%20function.&text=Note%20that%20axis%3D1%20here,the%20function%20to%20each%20column.

def create_address_uri(row):
    if pd.notna(row['full_address']):
        return str(URIRef(cw_namespace + row['full_address']))
    return pd.NA

df_cw['address_uri'] = df_cw.apply(create_address_uri, axis=1)

for uri in df_cw['address_uri'].dropna().unique():
    g.add((URIRef(uri), RDF.type, cw.Address))

In [40]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri
3955,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us
4233,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us
4385,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us
4391,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us
4397,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us


In [41]:
#number of triples after address uri
print(str(len(g)))

1428


##### URIs for City

same as address

In [42]:
unique_cities = df_cw['city'].dropna().unique()

In [43]:
df_cw['city_uri'] = pd.NA

def create_city_uri(row):
    if pd.notna(row['city']):
        return str(URIRef(cw_namespace + row['city']))
    return pd.NA

df_cw['city_uri'] = df_cw.apply(create_city_uri, axis=1)

for uri in df_cw['city_uri'].dropna().unique():
    g.add((URIRef(uri), RDF.type, cw.City))

In [44]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri,city_uri
3955,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles
4233,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles
4385,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles
4391,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles
4397,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles


In [45]:
#number of triples after city uri
print(str(len(g)))

1560


##### URIs for State

same as address

In [46]:
unique_states = df_cw['state'].dropna().unique()

In [47]:
df_cw['state_uri'] = pd.NA

def create_state_uri(row):
    if pd.notna(row['state']):
        return str(URIRef(cw_namespace + row['state']))
    return pd.NA

df_cw['state_uri'] = df_cw.apply(create_state_uri, axis=1)

for uri in df_cw['state_uri'].dropna().unique():
    g.add((URIRef(uri), RDF.type, cw.State))

In [48]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri,city_uri,state_uri
3955,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz
4233,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz
4385,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz
4391,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz
4397,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles

In [49]:
#number of triples after state uri
print(str(len(g)))

1634


##### URIs for Country

same as address

In [50]:
unique_countries = df_cw['country'].dropna().unique()

In [51]:
df_cw['country_uri'] = pd.NA

def create_country_uri(row):
    if pd.notna(row['country']):
        return str(URIRef(cw_namespace + row['country']))
    return pd.NA

df_cw['country_uri'] = df_cw.apply(create_country_uri, axis=1)

for uri in df_cw['country_uri'].dropna().unique():
    g.add((URIRef(uri), RDF.type, cw.Country))

In [52]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri,city_uri,state_uri,country_uri
3955,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us
4233,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us
4385,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us
4391,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us
4397,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,http://www.se

In [53]:
#number of triples after country uri
print(str(len(g)))

1635


##### URIs for restaurant type / categories

similar to menu item

In [54]:
#this was a bit tricky as needed to get the restaurant type to be the uri matched on the kind of pizza place
#kept on getting it stored incorrectly
#eventually figured out how to store it as a dictionary
#documentation: https://stackoverflow.com/questions/64431367/how-to-iterate-and-store-variables-in-a-dictionary

uri_class = {}
df_cw['restaurant_uri'] = pd.NA

def create_restaurant_uri(row):
    for class_name in filtered_restaurant_types_names:
        transformed_class_name = transform_class_name(class_name)
        if pd.notna(row['categories']) and transformed_class_name in row['categories']:
            uri = str(URIRef(cw_namespace + row['name_city']))
            uri_class[uri] = class_name
            return uri
    return pd.NA

df_cw['restaurant_uri'] = df_cw.apply(create_restaurant_uri, axis=1)

for uri in df_cw['restaurant_uri'].dropna().unique():
    if uri in uri_class:
        g.add((URIRef(uri), RDF.type, cw[uri_class[uri]]))

In [55]:
filtered_df = df_cw[df_cw['name_menu_item'] == 'bravo_pizza_hollywood_hawaiian_pizza']
filtered_df.tail(5)

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri,city_uri,state_uri,country_uri,restaurant_uri
3955,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_los_angeles
4233,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#pineapple,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_los_angeles
4385,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,NaN,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_los_angeles
4391,bravo_pizza_hollywood,5142_hollywood_blvd,los_angeles,us,90027.0,los_feliz,pizza_place,Hawaiian Pizza,11.99,usd,canadian_bacon_pineapple,bravo_pizza_hollywood_los_angeles,bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bacon,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_hawaiian_pizza,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#usd,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#11.99,bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bravo_pizza_hollywood_5142_hollywood_blvd_los_angeles_los_feliz_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_feliz,http://www.semanticweb.org/city/in3067-inm713/2024/restaura

In [56]:
#number of triples after categories uri

print(str(len(g)))

1742


##### URIs for postcode

In [57]:
#df_cw['postcode_uri'] = pd.NA
#additional_rows = []

#item_value_class_name = 'ItemValue'

#for idx, row in df_cw.iterrows():
#    if pd.notna(row['item value']) and isinstance(row['item value'], float):
#        item_value_uri = URIRef(cw_namespace + str(row['item value']))
#        new_row = row.copy()
#        new_row['item_value_uri'] = str(item_value_uri)
#        additional_rows.append(new_row)
#        g.add((item_value_uri, RDF.type, URIRef(cw_namespace + item_value_class_name)))

#if additional_rows:
#    df_cw = pd.concat([df_cw, pd.DataFrame(additional_rows)], ignore_index=True)

#### URIs and Triples for Object Properties

these have been made from the examples in the turtle file with data

just followed the individuals in there and saw what properties they were connected with

those have been built here

In [58]:
df_cw

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,name_city,name_menu_item,ingredient_uri,menu_item_uri,currency_uri,item_value_uri,full_address,address_uri,city_uri,state_uri,country_uri,restaurant_uri
0,little_pizza_paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Bianca Pizza,22.50,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_bianca_pizza,NaN,NaN,NaN,NaN,little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bend,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#or,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#little_pizza_paradise_bend
1,little_pizza_paradise,cascade_village_mall_across_from_target,bend,us,97701.0,or,pizza_place,Cheese Pizza,18.95,usd,NaN,little_pizza_paradise_bend,little_pizza_paradise_cheese_pizza,NaN,NaN,NaN,NaN,little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#little_pizza_paradise_cascade_village_mall_across_from_target_bend_or_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#bend,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#or,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#little_pizza_paradise_bend
2,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Margherita",12.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_margherita,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#brentwood,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_los_angeles
3,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Mushroom",13.00,usd,NaN,the_brentwood_los_angeles,the_brentwood_pizza_mushroom,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#brentwood,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_los_angeles
4,the_brentwood,148_s_barrington_ave,los_angeles,us,90049.0,brentwood,american_restaurant_bar_bakery,"Pizza, Puttenesca",13.00,usd,olives_onions_capers_tomatoes,the_brentwood_los_angeles,the_brentwood_pizza_puttenesca,NaN,NaN,NaN,NaN,the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_148_s_barrington_ave_los_angeles_brentwood_us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#los_angeles,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#brentwood,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#us,http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_brentwood_los_angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,zios_italian_kitchen_olathe,11981_s_strang_line_rd,olathe,us,66062.0,ks,italian_restaurant,Chicken Club Pizza,8.79,usd,homestyle_tomato_sauce_sl

In [59]:
#getting the distinct object property lists as we did with classes

distinct_object_properties_query = """
SELECT DISTINCT ?property
WHERE {
    ?property a owl:ObjectProperty .
}
"""

distinct_properties = g.query(distinct_object_properties_query)

In [60]:
distinct_properties

In [61]:
print("Raw output of the object properties query:")
for result in distinct_properties:
    print(result)

Raw output of the object properties query:
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#locatedIn'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#amountCurrency'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#hasIngredient'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#servesMenuItem'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#containsState'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#containsAdress'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#locatedInState'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#locatedInCity'),)
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#containsLocation'),)
(rdflib.term.URIRef('http://www.sema

In [62]:
distinct_property_names = [properties[0].split('#')[-1] for properties in distinct_properties]

In [63]:
distinct_property_names

['locatedIn',
 'amountCurrency',
 'hasIngredient',
 'servesMenuItem',
 'containsState',
 'containsAdress',
 'locatedInState',
 'locatedInCity',
 'containsLocation',
 'hasValue',
 'containsCity',
 'locatedInAddress',
 'servedInRestaurant',
 'isIngredientOf',
 'locatedInCountry',
 'servedIn',
 'serves']

##### Object Property: amountCurrency

In [64]:
def amount_currency_property(row):
    #checking for NAs and filtering out as many rows were created during uri creation so those are not necessary
    if pd.notna(row['item_value_uri']) and pd.notna(row['currency_uri']):
        item_value_uri = URIRef(row['item_value_uri'])
        currency_uri = URIRef(row['currency_uri'])
        g.add((item_value_uri, cw.amountCurrency, currency_uri))

df_cw.apply(amount_currency_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [65]:
#number of triples after amount currency property

print(str(len(g)))

1872


##### Object Property: locatedInCity

In [66]:
#this is for address

def address_in_city_property(row):
    if pd.notna(row['address_uri']) and pd.notna(row['city_uri']):
        address_uri = URIRef(row['address_uri'])
        city_uri = URIRef(row['city_uri'])
        g.add((address_uri, cw.locatedInCity, city_uri))

df_cw.apply(address_in_city_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [67]:
#number of triples after located_in_city_property

print(str(len(g)))

2027


In [68]:
#this is for restaurant

def restaurant_in_city_property(row):
    if pd.notna(row['restaurant_uri']) and pd.notna(row['city_uri']):
        restaurant_uri = URIRef(row['restaurant_uri'])
        city_uri = URIRef(row['city_uri'])
        g.add((restaurant_uri, cw.locatedInCity, city_uri))

df_cw.apply(restaurant_in_city_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [69]:
#number of triples after located_in_city_property

print(str(len(g)))

2134


##### Object Property: hasIngredient

In [70]:
def has_ingredient_property(row):
    if pd.notna(row['menu_item_uri']) and pd.notna(row['ingredient_uri']):
        menu_item_uri = URIRef(row['menu_item_uri'])
        ingredient_uri = URIRef(row['ingredient_uri'])
        g.add((menu_item_uri, cw.hasIngredient, ingredient_uri))

df_cw.apply(has_ingredient_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [71]:
#number of triples after has ingredient_property

print(str(len(g)))

2293


##### Object Property: hasValue

In [72]:
#def has_value_property(row):
#    if pd.notna(row['menu_item_uri']) and pd.notna(row['item_value_uri']):
#        menu_item_uri = URIRef(row['menu_item_uri'])
#        item_value_uri = URIRef(row['item_value_uri'])
#        g.add((menu_item_uri, cw.hasValue, item_value_uri))

#df_cw.apply(has_value_property, axis=1)

In [73]:
def has_value_property(row):
    if pd.notna(row['menu_item_uri']) and pd.notna(row['item_value_uri']):
        menu_item_uri = URIRef(row['menu_item_uri'])
        item_value = float(row['item value'])
        item_value_literal = Literal(item_value, datatype=XSD.double)
        #item_value_uri = URIRef(row['item_value_uri'])
        g.add((menu_item_uri, cw.hasValue, item_value_literal))

df_cw.apply(has_value_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [74]:
#number of triples after has value_property

print(str(len(g)))

2396


##### Object Property: locatedInAddress

In [75]:
def located_in_address_property(row):
    if pd.notna(row['restaurant_uri']) and pd.notna(row['address_uri']):
        restaurant_uri = URIRef(row['restaurant_uri'])
        address_uri = URIRef(row['address_uri'])
        g.add((restaurant_uri, cw.locatedInAddress, address_uri))

df_cw.apply(located_in_address_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [76]:
#number of triples after located_in_address_property

print(str(len(g)))

2503


##### Object Property: serves

In [77]:
def serves_property(row):
    if pd.notna(row['restaurant_uri']) and pd.notna(row['menu_item_uri']):
        restaurant_uri = URIRef(row['restaurant_uri'])
        menu_item_uri = URIRef(row['menu_item_uri'])
        g.add((restaurant_uri, cw.serves, menu_item_uri))

df_cw.apply(serves_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [78]:
#number of triples after serves_property

print(str(len(g)))

2598


##### Object Property: locatedInState

In [79]:
def located_in_state_property(row):
    if pd.notna(row['city_uri']) and pd.notna(row['state_uri']):
        city_uri = URIRef(row['city_uri'])
        state_uri = URIRef(row['state_uri'])
        g.add((city_uri, cw.locatedInState, state_uri))

df_cw.apply(located_in_state_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [80]:
#number of triples after located_in_state_property

print(str(len(g)))

2738


##### Object Property: locatedInCountry

In [81]:
def located_in_country_property(row):
    if pd.notna(row['country_uri']) and pd.notna(row['state_uri']):
        country_uri = URIRef(row['country_uri'])
        state_uri = URIRef(row['state_uri'])
        g.add((state_uri, cw.locatedInCountry, country_uri))

df_cw.apply(located_in_country_property, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [82]:
#number of triples after located_in_country_property

print(str(len(g)))

2812


#### Now we add some Data Properties

using data property restaurantName and name

In [83]:
#lets add the literal names for restaurants

def restaurant_names_data(row):
    if pd.notna(row['restaurant_uri']) and pd.notna(row['name_city']):
        restaurant_uri = URIRef(row['restaurant_uri'])
        restaurant_name = Literal(row['name_city'], datatype=XSD.string)

        g.add((restaurant_uri, cw.restaurantName, restaurant_name))

df_cw.apply(restaurant_names_data, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [84]:
#lets add the literal names for city

def city_names_data(row):
    if pd.notna(row['city_uri']) and pd.notna(row['city']):
        city_uri = URIRef(row['city_uri'])
        city_name = Literal(row['city'], datatype=XSD.string)

        g.add((city_uri, cw.name, city_name))

df_cw.apply(city_names_data, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [85]:
#lets add the literal names for state

def state_names_data(row):
    if pd.notna(row['state_uri']) and pd.notna(row['state']):
        state_uri = URIRef(row['state_uri'])
        state_name = Literal(row['state'], datatype=XSD.string)

        g.add((state_uri, cw.name, state_name))

df_cw.apply(state_names_data, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
4639    None
4640    None
4641    None
4642    None
4643    None
Length: 4644, dtype: object

In [86]:
#number of triples after value data_property

print(str(len(g)))

3125


In [87]:
#checking for vegan pizzas

vega_query = """
SELECT ?veganPizza
WHERE {
  ?veganPizza rdf:type cw:VeganPizza .
}
"""

vegans = g.query(vega_query)

print("Vegan Pizzas:")
for vegan in vegans:
    print(vegan.veganPizza)

Vegan Pizzas:
http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#broad_street_cafe_vegan_pizza
http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#johns_of_12th_street_vegan_margherita_pizza


In [88]:
output = 'generated_rdf.ttl'
g.serialize(destination=output, format='turtle')

### Subtask RDF.3

#### Subtask RDF.3.2 Use Wikidata KG

the output is odd. i tried to change the string cleaning for country column to US, USA, America and a few others from https://www.wikidata.org/wiki/ but still was not able to get America correctly

In [89]:
class CW1(object):
    '''
    Example of a partial solution for Lab 5 - taken from lab 5 
    '''
    def __init__(self, input_file):
           
        self.file = input_file
    
        #Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
        self.stringToURI = dict()
        
        
        #1. GRAPH INITIALIZATION
    
        #Empty graph
        self.g1 = Graph()
        
        #Note that this is the same namespace used in the ontology "ontology_lab5.ttl"
        self.cw_namespace1= "http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#"
        
        #Special namspaces class to create directly URIRefs in python.           
        self.cw1 = Namespace(self.cw_namespace1)
        
        #Prefixes for the serialization
        self.g1.bind("cw1", self.cw1) #lab5 is a newly created namespace
        self.g1.bind("wiki", "https://www.wikidata.org/w/api.php") 
        
        #Load data in dataframe  
        self.data_frame = pd.read_csv(self.file)    
    
        
        #KG
        self.wikidata = WikidataAPI()
    
    
    
    def performTask1(self):
        self.CovertCSVToRDF(False)
        
    def performTask2(self):
        self.CovertCSVToRDF(True)
            

    def CovertCSVToRDF(self, useExternalURI):
                 
        #In a large ontology one would need to find a more automatic way to use the ontology vocabulary. 
        #E.g.,  via matching. In a similar way as we match entities to a large KG like DBPedia or Wikidata
        #Since we are dealing with very manageable ontologies, we can integrate their vocabulary 
        #within the code. E.g.,: lab5.City
        
        
        #We modularize the transformation to RDF. The transformation is tailored to the given table, but 
        #the individual components/mappings are relatively generic (especially type and literal triples).
        
        #Mappings may required one or more columns as input and create 1 or more triples for an entity
        
        
        if 'country' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('country', self.cw1.Country, useExternalURI)
                    
        if 'city' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('city', self.cw1.City, useExternalURI)
                    
        if 'state' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('state', self.cw1.State, useExternalURI)

        
    
    def processLexicalName(self, name):
        #Remove potential spaces and other characters not allowed in URIs
        
        #This method may need to be extended
        #Other problematic characters: 
        #{", "}", "|", "\", "^", "~", "[", "]", and "`"
        return name.replace(" ", "_").replace("(", "").replace(")", "")
                
          
    def createURIForEntity(self, name, useExternalURI):
        
        #We create fresh URI (default option)
        self.stringToURI[name] = self.cw_namespace1 + self.processLexicalName(name)
        
        if useExternalURI: #We connect to online KG
            uri = self.getExternalKGURI(name)
            if uri!="":
                self.stringToURI[name]=uri
        
        return self.stringToURI[name]
    
    
        
    def getExternalKGURI(self, name):
        '''
        Approximate solution: We get the entity with highest lexical similarity
        The use of context may be necessary in some cases        
        '''
        
        entities = self.wikidata.getKGEntities(name, 10)
        current_sim = -1
        current_uri=''
        for ent in entities:           
            isub_score = isub(name, ent.label) 
            if current_sim < isub_score:
                current_uri = ent.ident
                current_sim = isub_score
        
            #print(current_uri)
        return current_uri 
            

    def mappingToCreateTypeTriple(self, subject_column, class_type, useExternalURI):
        
        for subject in self.data_frame[subject_column]:
                
            #We use the ascii name to create the fresh URI for a city in the dataset
            if subject.lower() in self.stringToURI:
                entity_uri=self.stringToURI[subject.lower()]
            else:
                entity_uri=self.createURIForEntity(subject.lower(), useExternalURI)
            
            #TYPE TRIPLE
            #For the individuals we use URIRef to create an object "URI" out of the string URIs
            #For the concepts we use the ones in the ontology and we are using the NameSpace class
            #Alternatively one could use URIRef(self.lab5_ns_str+"City") for example 
            self.g1.add((URIRef(entity_uri), RDF.type, class_type))
    

    def is_nan(self, x):
        return (x != x)

    def printGraph(self):
        print(self.g1.serialize(format="turtle").decode("utf-8"))        
    
    def saveGraph(self, file_output):
        
        ##SAVE/SERIALIZE GRAPH
        #print(self.g.serialize(format="turtle").decode("utf-8"))
        self.g1.serialize(destination=file_output, format='turtle')

In [90]:
file = "cw_data.csv"
output_file_name = "wikidata_rdf"

In [91]:
solution = CW1(file)

In [92]:
task = "task2"

In [93]:
solution.performTask2()

In [94]:
#file = "cw_data.csv"
#solution = CW1(file)
#solution.performTask2()
solution.printGraph()

@prefix cw1: <http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#> .

cw1:city_of_spokane_valley a cw1:State .

cw1:no_providence a cw1:State .

cw1:s_connelsvl a cw1:State .

cw1:village_of_wellington a cw1:State .

<http://www.wikidata.org/entity/Q101001102> a cw1:State .

<http://www.wikidata.org/entity/Q1010244> a cw1:City .

<http://www.wikidata.org/entity/Q1015021> a cw1:City .

<http://www.wikidata.org/entity/Q104320373> a cw1:State .

<http://www.wikidata.org/entity/Q1049772> a cw1:State .

<http://www.wikidata.org/entity/Q1053> a cw1:State .

<http://www.wikidata.org/entity/Q1065526> a cw1:City .

<http://www.wikidata.org/entity/Q1066487> a cw1:City .

<http://www.wikidata.org/entity/Q1072657> a cw1:City,
        cw1:State .

<http://www.wikidata.org/entity/Q1084539> a cw1:City .

<http://www.wikidata.org/entity/Q1084657> a cw1:City .

<http://www.wikidata.org/entity/Q1086321> a cw1:City .

<http://www.wikidata.org/entity/Q11071485> a cw1:State .

<http://www.wikid

In [95]:
solution.saveGraph(output_file_name)

#### Subtask RDF.3.1 Use Google KG

same code as wiki data just small changes in the lookup

In [96]:
class CW2(object):
    '''
    Example of a partial solution for Lab 5 - taken from lab 5 
    '''
    def __init__(self, input_file):
           
        self.file = input_file
    
        #Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
        self.stringToURI = dict()
        
        
        #1. GRAPH INITIALIZATION
    
        #Empty graph
        self.g2 = Graph()
        
        #Note that this is the same namespace used in the ontology "ontology_lab5.ttl"
        self.cw_namespace2= "http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#"
        
        #Special namspaces class to create directly URIRefs in python.           
        self.cw2 = Namespace(self.cw_namespace2)
        
        #Prefixes for the serialization
        self.g2.bind("cw2", self.cw2) #lab5 is a newly created namespace
        self.g2.bind("google", "https://kgsearch.googleapis.com/v1/entities:search") 
        
        #Load data in dataframe  
        self.data_frame = pd.read_csv(self.file)    
    
        
        #KG
        self.kg = GoogleKGLookup()
    
    
    
    def performTask1(self):
        self.CovertCSVToRDF(False)
        
    def performTask2(self):
        self.CovertCSVToRDF(True)
            

    def CovertCSVToRDF(self, useExternalURI):
                 
        #In a large ontology one would need to find a more automatic way to use the ontology vocabulary. 
        #E.g.,  via matching. In a similar way as we match entities to a large KG like DBPedia or Wikidata
        #Since we are dealing with very manageable ontologies, we can integrate their vocabulary 
        #within the code. E.g.,: lab5.City
        
        
        #We modularize the transformation to RDF. The transformation is tailored to the given table, but 
        #the individual components/mappings are relatively generic (especially type and literal triples).
        
        #Mappings may required one or more columns as input and create 1 or more triples for an entity
        
        
        if 'country' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('country', self.cw2.Country, useExternalURI)
                    
        if 'city' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('city', self.cw2.City, useExternalURI)
                    
        if 'state' in self.data_frame:
            
            #We give subject column and target type
            self.mappingToCreateTypeTriple('state', self.cw2.State, useExternalURI)

        
    
    def processLexicalName(self, name):
        #Remove potential spaces and other characters not allowed in URIs
        
        #This method may need to be extended
        #Other problematic characters: 
        #{", "}", "|", "\", "^", "~", "[", "]", and "`"
        return name.replace(" ", "_").replace("(", "").replace(")", "")
                
          
    def createURIForEntity(self, name, useExternalURI):
        
        #We create fresh URI (default option)
        self.stringToURI[name] = self.cw_namespace2 + self.processLexicalName(name)
        
        if useExternalURI: #We connect to online KG
            uri = self.getExternalKGURI(name)
            if uri!="":
                self.stringToURI[name]=uri
        
        return self.stringToURI[name]
    
    
        
    def getExternalKGURI(self, name):
        '''
        Approximate solution: We get the entity with highest lexical similarity
        The use of context may be necessary in some cases        
        '''
        
        entities = self.kg.getKGEntities(name, 10)
        current_sim = -1
        current_uri=''
        for ent in entities:           
            isub_score = isub(name, ent.label) 
            if current_sim < isub_score:
                current_uri = ent.ident
                current_sim = isub_score
        
            #print(current_uri)
        return current_uri 
            

    def mappingToCreateTypeTriple(self, subject_column, class_type, useExternalURI):
        
        for subject in self.data_frame[subject_column]:
                
            #We use the ascii name to create the fresh URI for a city in the dataset
            if subject.lower() in self.stringToURI:
                entity_uri=self.stringToURI[subject.lower()]
            else:
                entity_uri=self.createURIForEntity(subject.lower(), useExternalURI)
            
            #TYPE TRIPLE
            #For the individuals we use URIRef to create an object "URI" out of the string URIs
            #For the concepts we use the ones in the ontology and we are using the NameSpace class
            #Alternatively one could use URIRef(self.lab5_ns_str+"City") for example 
            self.g2.add((URIRef(entity_uri), RDF.type, class_type))
    

    def is_nan(self, x):
        return (x != x)

    def printGraph(self):
        print(self.g2.serialize(format="turtle").decode("utf-8"))        
    
    def saveGraph(self, file_output):
        
        ##SAVE/SERIALIZE GRAPH
        #print(self.g.serialize(format="turtle").decode("utf-8"))
        self.g2.serialize(destination=file_output, format='turtle')

In [97]:
file = "cw_data.csv"
output_file_name = "google_rdf"

In [98]:
solution = CW2(file)

In [99]:
task = "task2"

In [100]:
solution.performTask2()

In [101]:
solution.saveGraph(output_file_name)

### Subtask RDF.4 

reasoning code taken from lab

In [102]:
 def performReasoning(ontology_file):
        
        print("Data triples from CSV: '" + str(len(g)) + "'.")
    
        g.load(ontology_file,  format=guess_format(ontology_file))
        
        print("Triples including ontology: '" + str(len(g)) + "'.")   

        owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(g)
        
        print("Triples after OWL 2 RL reasoning: '" + str(len(g)) + "'.")

In [103]:
performReasoning('pizza-restaurants-ontology.ttl')

Data triples from CSV: '3125'.
Triples including ontology: '3547'.
Triples after OWL 2 RL reasoning: '25361'.


In [104]:
output = 'extended_generated_rdf.ttl'
g.serialize(destination=output, format='turtle')

## 2.3 SPARQL and Reasoning (Task SPARQL)

### Subtask SPARQL.1 Create a query with at least two triple pattern and a FILTER

creating a query that simply filters for pizza places that serve greek pizza, specically original girogios

In [105]:
query_sparql_1 = """
SELECT ?restaurant ?menuitem ?value
WHERE {
    ?restaurant rdf:type cw:PizzaPlace;
        cw:serves ?menuitem;
        cw:restaurantName ?name .
      ?menuitem rdf:type cw:GreekPizza;
        cw:hasValue ?value .

    FILTER (str(?name) = "original_giorgios_cockeysville")
}
"""

query_sparql_1_results = g.query(query_sparql_1)

print("Query Results:")
for row in query_sparql_1_results:
    print(row)

Query Results:
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#original_giorgios_cockeysville'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#original_giorgios_greek_pizza'), rdflib.term.Literal('16.95', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))


In [106]:
#saving as csv
#documentation: https://www.youtube.com/watch?v=FD_xFFotvxU

g.query(query_sparql_1).serialize(destination='query_sparql_1_results.csv', format='csv')

### Subtask SPARQL.2 Create a query that uses at least three triple pattern, a FILTER and a Function

creating a query that filters all restaurants that serve pizza with a value of greater than 15 and we are returning the rounded value

In [107]:
query_sparql_2 = """
SELECT ?restaurant ?menuitem (ROUND(?value) as ?roundedValue)
WHERE {
    ?restaurant rdf:type cw:Restaurant;
        cw:serves ?menuitem;
        cw:restaurantName ?name .
    ?menuitem rdf:type cw:Pizza;
        cw:hasValue ?value .

    FILTER (?value > 15.0)
}
"""


query_sparql_2_results = g.query(query_sparql_2)

print("Query Results:")
for row in query_sparql_2_results:
    print(row)

Query Results:
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_indianapolis'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_gourmet_chicken_ranch_pizza'), rdflib.term.Literal('19', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#joeys_n_y_pizza_henderson'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#joeys_n_y_pizza_supreme_pizza'), rdflib.term.Literal('22', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#ninos_trattoria_pizzeria_lawrence_township'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#ninos_trattoria_pizzeria_chicken_alfredo_pizza'), rdflib.term.Literal('20', datatype=rdflib.term.UR

In [108]:
g.query(query_sparql_2).serialize(destination='query_sparql_2_results.csv', format='csv')

### Subtask SPARQL.3 Create a query that uses the Union graph pattern and SPARQL 1 negation

creating a query that negates query 1. essentially the negation says return restaurants that are serving greek pizza not called gregorios. we union this with a query that gives us pizza places that serve another pizza mexican pizza

the negation is in the filter. before we wanted a specific restaurant, but now we want everything but that

what we noted when adding the pizza types is happening here where the restaurant name is tagging chicken pizzas as greek pizzas as welln

In [109]:
query_sparql_3 = """
SELECT ?restaurant ?menuitem ?value
WHERE {
    {
        ?restaurant rdf:type cw:PizzaPlace;
                    cw:serves ?menuitem;
                    cw:restaurantName ?name .
        ?menuitem rdf:type cw:GreekPizza;
                  cw:hasValue ?value .
        FILTER (str(?name) != "original_giorgios_cockeysville")
    }
    UNION
    {
        ?restaurant rdf:type cw:PizzaPlace;
                    cw:serves ?menuitem .
        ?menuitem rdf:type cw:MexicanPizza;
                  cw:hasValue ?value .
    }
}
"""

query_sparql_3_results = g.query(query_sparql_3)

print("Query Results:")
for row in query_sparql_3_results:
    print(row)

Query Results:
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_indianapolis'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_cheese_pizza'), rdflib.term.Literal('1.42', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_indianapolis'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_gourmet_chicken_ranch_pizza'), rdflib.term.Literal('18.76', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_indianapolis'), rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_white_pizza'), rdflib.term.Literal('18.35', datatype=rdflib.term.URIRef('http://www.w3.org

In [110]:
g.query(query_sparql_3).serialize(destination='query_sparql_3_results.csv', format='csv')

### Subtask SPARQL.4 Create a query that groups results, uses aggregates, and a filter over the aggregates

here we are taking restaurants and their menu items and summing the value of the menu items to get a total price of the menu

we are then grouping by restaurant to get a total price of the restaurant

we are then filtering using having to get only those sums that are greater than 50s

In [111]:
query_sparql_4 = """
SELECT ?restaurant (SUM(?value) as ?totalprice)
WHERE {
    ?restaurant rdf:type cw:Restaurant;
                cw:serves ?menuitem;
                cw:restaurantName ?name .
    ?menuitem rdf:type cw:Pizza;
              cw:hasValue ?value .
}
GROUP BY ?restaurant
HAVING (SUM(?value) > 50.0)
"""


query_sparql_4_results = g.query(query_sparql_4)

print("Query Results:")
for row in query_sparql_4_results:
    print(row)

Query Results:
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#greeks_pizzeria_indianapolis'), rdflib.term.Literal('71.73000000000002', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#ninos_trattoria_pizzeria_lawrence_township'), rdflib.term.Literal('59.849999999999994', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#the_fire_den_alameda'), rdflib.term.Literal('167.93', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#abos_pizza_littleton'), rdflib.term.Literal('70.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#origin

In [112]:
g.query(query_sparql_4).serialize(destination='query_sparql_4_results.csv', format='csv')

### Subtask SPARQL.5 Create a query (different from SPARQL.4) that groups results, uses aggregates, filters the results over the aggregates and orders the results according to two variables


had to change the query a lot to speed it up. averages and travesing down to state was taking very long

we are getting the most expensive pizza at a restaurant in a city and showing only the prices that are more than 10 and we are showing only the top 5 restaurants with their cityes

In [113]:
query_sparql_5 = """
SELECT ?cityname ?restaurantname (MAX(?value) AS ?maxpizzaprice)
WHERE {
    ?restaurant rdf:type cw:Restaurant;
                cw:serves ?menuitem;
                cw:restaurantName ?restaurantname;
                cw:locatedInAddress ?address.
    ?address cw:locatedInCity ?city.
    ?city rdf:type cw:City;
          cw:name ?cityname.
    ?menuitem rdf:type cw:Pizza;
              cw:hasValue ?value.
}
GROUP BY ?cityname ?restaurantname
HAVING (MAX(?value) > 10)
ORDER BY DESC(MAX(?value)) ?cityname
LIMIT 5
"""


query_sparql_5_results = g.query(query_sparql_5)

print("Query Results:")
for row in query_sparql_5_results:
    print(row)

Query Results:
(rdflib.term.Literal('alameda', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('the_fire_den_alameda', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('23.99', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.Literal('littleton', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('abos_pizza_littleton', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('23.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.Literal('bend', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('little_pizza_paradise_bend', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('22.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.te

In [114]:
g.query(query_sparql_5).serialize(destination='query_sparql_5_results.csv', format='csv')

## 2.4 Ontology Alignment (Task OA)

using code from lab 8

### Subtask OA.1

#### Class Equivalence

In [115]:
#defining functions for getting the onotology data

def getClasses(onto):        
    return onto.classes()
    
def getDataProperties(onto):        
    return onto.data_properties()
    
def getObjectProperties(onto):        
    return onto.object_properties()
    
def getIndividuals(onto):    
    return onto.individuals()

def getRDFSLabelsForEntity(entity):
    return entity.label

def getRDFSLabelsForEntity(entity):
    return entity.label 

def loadOntology_Classes(uri_onto):
    
    onto = get_ontology(uri_onto).load()
    
    print("Classes in Ontology: " + str(len(list(getClasses(onto)))))
    for cls in getClasses(onto):             
        #print(cls.iri)
        print("\t"+cls.name)  
        #Labels from RDFS label
        #print("\t"+str(getRDFSLabelsForEntity(cls)))

#creating another function to save the loaded ontology as a list to use for preproessing
def onotology_class_list(uri_onto):
    onto = get_ontology(uri_onto).load()
    class_names = [cls.name for cls in onto.classes()]
    return class_names

#creating another function to save the loaded ontology as a list to use for preproessing. this time for objects
def onotology_object_property_list(uri_onto):
    onto = get_ontology(uri_onto).load()
    object_property_names = [obj.name for obj in onto.object_properties()]
    return object_property_names

In [116]:
pizza_uri="pizza.owl"
cw_uri = "pizza-restaurants-ontology.owl"

In [117]:
pizza_classes_print = loadOntology_Classes(pizza_uri)

Classes in Ontology: 100
	Pizza
	PizzaBase
	Food
	Spiciness
	FoodTopping
	American
	NamedPizza
	MozzarellaTopping
	PeperoniSausageTopping
	TomatoTopping
	AmericanHot
	HotGreenPepperTopping
	JalapenoPepperTopping
	AnchoviesTopping
	FishTopping
	ArtichokeTopping
	VegetableTopping
	Mild
	AsparagusTopping
	Cajun
	OnionTopping
	PeperonataTopping
	PrawnsTopping
	TobascoPepperSauce
	CajunSpiceTopping
	HerbSpiceTopping
	Hot
	RosemaryTopping
	CaperTopping
	Capricciosa
	HamTopping
	OliveTopping
	Caprina
	GoatsCheeseTopping
	SundriedTomatoTopping
	CheeseTopping
	PizzaTopping
	CheeseyPizza
	CheeseyVegetableTopping
	ChickenTopping
	MeatTopping
	Country
	DomainConcept
	DeepPanBase
	ThinAndCrispyBase
	ValuePartition
	Fiorentina
	GarlicTopping
	ParmesanTopping
	SpinachTopping
	FourCheesesTopping
	FourSeasons
	MushroomTopping
	FruitTopping
	FruttiDiMare
	MixedSeafoodTopping
	Medium
	Giardiniera
	LeekTopping
	PetitPoisTopping
	SlicedTomatoTopping
	GorgonzolaTopping
	GreenPepperTopping
	PepperTopping
	Ho

In [118]:
cw_classes_print = loadOntology_Classes(cw_uri)

Classes in Ontology: 151
	ItemValue
	Currency
	Address
	City
	Location
	State
	Food
	Ingredient
	Country
	AmericanPizza
	PizzaByStyle
	AmericanRestaurant
	Restaurant
	Anchovies
	SeaFood
	Artichokes
	Vegetable
	AsianRestaurant
	Bacon
	Meat
	Bakery
	Bar
	BarsPubsClubs
	BarAndGrill
	BarbecuePizza
	BarbecueSauce
	Pizza
	PizzaByIngredient
	Sauce
	VeganIngredient
	Basil
	Herbs
	Beans
	Seeds
	BeansPizza
	Beef
	BeerPlace
	BlackOlives
	Olives
	BlueCheese
	Cheese
	Broccoli
	BurgerPlace
	ButternutSquash
	WinterSquash
	CalifornianPizza
	Capers
	Carrot
	RootVegetable
	Cheddar
	VegetarianIngredient
	CherryTomato
	Tomato
	Chicken
	ChickenPizza
	ChineseRestaurant
	Chorizo
	Club
	CocktailBars
	CoffeeShop
	CrabMeat
	DietaryRestaurant
	Eggplant
	Feta
	FetaPizza
	Fig
	Fruit
	FrenchRestaurant
	MediterraneanRestaurant
	FruitPizza
	Garlic
	Gastropub
	Pub
	GlutenFreeRestaurant
	GoatCheese
	Gorgonzola
	GourmetRestaurants
	GreekPizza
	Spinach
	GreekRestaurant
	GreenOlives
	GreenPepper
	Pepper
	Ham
	HawaiianPizz

In [119]:
pizza_classes = onotology_class_list(pizza_uri)
cw_classes = onotology_class_list(cw_uri)

In [120]:
#defining a function that will transform the class list from cw_onto to resemble pizza
#classes are already like proper case with no spaces
#restaurants and locations and pizzas wont get matched
#but we can add Topping so that some of the toppings get matched
#we are ignoring things that end with pizza restuarnta or country as those are not in pizza owl
#we are adding tuples because when i made the similarity function i realized the orignal class shoud be quivalent not the transformed

def process_class_names(classes):
    modified_classes = []
    for class_name in classes:
        if class_name.endswith('Pizza') or class_name.endswith('Restaurant') or class_name.endswith('Country'):
            modified_classes.append((class_name, class_name))
        else:
            modified_class_name = class_name + 'Topping'
            modified_classes.append((class_name, modified_class_name))
    return modified_classes

In [121]:
transformed_cw_onto_classes = process_class_names(cw_classes)

In [122]:
transformed_cw_onto_classes

[('ItemValue', 'ItemValueTopping'),
 ('Currency', 'CurrencyTopping'),
 ('Address', 'AddressTopping'),
 ('City', 'CityTopping'),
 ('Location', 'LocationTopping'),
 ('State', 'StateTopping'),
 ('Food', 'FoodTopping'),
 ('Ingredient', 'IngredientTopping'),
 ('Country', 'Country'),
 ('AmericanPizza', 'AmericanPizza'),
 ('PizzaByStyle', 'PizzaByStyleTopping'),
 ('AmericanRestaurant', 'AmericanRestaurant'),
 ('Restaurant', 'Restaurant'),
 ('Anchovies', 'AnchoviesTopping'),
 ('SeaFood', 'SeaFoodTopping'),
 ('Artichokes', 'ArtichokesTopping'),
 ('Vegetable', 'VegetableTopping'),
 ('AsianRestaurant', 'AsianRestaurant'),
 ('Bacon', 'BaconTopping'),
 ('Meat', 'MeatTopping'),
 ('Bakery', 'BakeryTopping'),
 ('Bar', 'BarTopping'),
 ('BarsPubsClubs', 'BarsPubsClubsTopping'),
 ('BarAndGrill', 'BarAndGrillTopping'),
 ('BarbecuePizza', 'BarbecuePizza'),
 ('BarbecueSauce', 'BarbecueSauceTopping'),
 ('Pizza', 'Pizza'),
 ('PizzaByIngredient', 'PizzaByIngredientTopping'),
 ('Sauce', 'SauceTopping'),
 ('Vega

In [123]:
#still need to match things like OliveTopping to OlivesTopping - cant chop the s off as then anchovies will become incorrect
#i had trouble importing levenshiten so found an alternative scoreer
#documentation: https://stackoverflow.com/questions/17388213/find-the-similarity-metric-between-two-strings
#documentation: https://docs.python.org/3/library/difflib.html#sequencematcher-examples

#creating a function for the similarity score
def similarity_score(list1, list2):
    results = []
    for item in list1:
        for index in list2:
            item1 = index[0]
            item2 = index[1]
            similarity = difflib.SequenceMatcher(None, item, item2).ratio()
            #not every score is important
            #non veg and veg had a score of 0.91 so making this higher than that to avoid close calls
            if similarity >= 0.95:
                results.append((item, item2, item1, similarity))

    equivalent_classes_df = pd.DataFrame(results, columns=['List1', 'List2 Transformed', 'List2 Original', 'Similarity Score - 1 and Transformed'])
    return equivalent_classes_df

In [124]:
equivalent_classes_df = similarity_score(pizza_classes, transformed_cw_onto_classes)

In [125]:
equivalent_classes_df.rename(columns={
    'List1': 'Pizza_Classes',
    'List2 Transformed': 'Transformed_CW_Onto_Classes',
    'List2 Original': 'Original_CW_Onto_Classes',
    'Similarity Score - 1 and Transformed': 'Similarity Score - Pizza Classes to Transformed CW Onto Classes'
}, inplace=True)

In [126]:
#these are the classes that will get the equivalence
#we ar going to filter out the second row as Food is not the same as Food Topping
equivalent_classes_df = equivalent_classes_df.drop(1)
equivalent_classes_df

,Pizza_Classes,Transformed_CW_Onto_Classes,Original_CW_Onto_Classes,Similarity Score - Pizza Classes to Transformed CW Onto Classes
0,Pizza,Pizza,Pizza,1.000000
2,NamedPizza,NamedPizza,NamedPizza,1.000000
3,MozzarellaTopping,MozzarellaTopping,Mozzarella,1.000000
4,TomatoTopping,TomatoTopping,Tomato,1.000000
5,JalapenoPepperTopping,JalapenoPepperTopping,JalapenoPepper,1.000000
6,AnchoviesTopping,AnchoviesTopping,Anchovies,1.000000
7,ArtichokeTopping,ArtichokesTopping,Artichokes,0.969697
8,VegetableTopping,VegetableTopping,Vegetable,1.000000
9,OnionTopping,OnionTopping,Onion,1.000000
10,RosemaryTopping,RosemaryTopping,Rosemary,1.000000


#### Object Property Equivalence

In [127]:
pizza_object_properties = onotology_object_property_list(pizza_uri)
cw_object_properties = onotology_object_property_list(cw_uri)

In [128]:
cw_object_properties

['amountCurrency',
 'containsAdress',
 'containsLocation',
 'containsCity',
 'locatedIn',
 'containsState',
 'hasIngredient',
 'isIngredientOf',
 'hasValue',
 'locatedInAddress',
 'locatedInCity',
 'locatedInCountry',
 'locatedInState',
 'servedIn',
 'serves',
 'servedInRestaurant',
 'servesMenuItem']

In [129]:
pizza_object_properties

['hasBase',
 'hasIngredient',
 'isBaseOf',
 'hasCountryOfOrigin',
 'isIngredientOf',
 'hasSpiciness',
 'hasTopping',
 'isToppingOf']

In [130]:
#there are only 2 being matched here
#but just for code reusability, lets write a for loop to match and store things 
#i could have also said that hastopping is equivalent to has ingredient manually and that would have opened more sparql ability
#but pizza had ingredient and topping and i wasnt sure how they worked together so i didnt equate them

object_property_matches = []

for item1 in cw_object_properties:
    for item2 in pizza_object_properties:
        if item1 == item2:
            object_property_matches.append((item1, item2))
            
equivalent_object_properties_df = pd.DataFrame(object_property_matches, columns=['CW Object Properties', 'Pizza Object Properties'])

In [131]:
equivalent_object_properties_df

,CW Object Properties,Pizza Object Properties
0,hasIngredient,hasIngredient
1,isIngredientOf,isIngredientOf


#### Saving Equivalence to Graph

In [132]:
#lets initialize the graph and save the triples to the graph

g = Graph()

pizza_owl_namespace = 'http://www.co-ode.org/ontologies/pizza/pizza.owl#'
cw_namespace = 'http://www.semanticweb.org/city/in3067-inm713/2024/restaurants#'

city = Namespace(cw_namespace)
pizza = Namespace(pizza_owl_namespace)

g.bind('city', city)
g.bind('pizza', pizza)

In [133]:
for index, row in equivalent_classes_df.iterrows():
    class1 = URIRef(pizza_owl_namespace + row['Pizza_Classes'])
    class2 = URIRef(cw_namespace + row['Original_CW_Onto_Classes'])
    g.add((class2, OWL.equivalentClass, class1))

In [134]:
for index, row in equivalent_object_properties_df.iterrows():
    object_property1 = URIRef(pizza_owl_namespace + row['Pizza Object Properties'])
    object_property2 = URIRef(cw_namespace + row['CW Object Properties'])
    g.add((object_property2, OWL.equivalentProperty, object_property1))

In [135]:
#getting the graph and saving it
g.serialize(destination='pizza_cw_equivalences.ttl', format='turtle')

### Subtask OA.2

In [136]:
#code taken from lab 8

def compareWithReference(reference_mappings_file, system_mappings_file):
    ref_mappings = Graph()
    ref_mappings.parse(reference_mappings_file, format="ttl")
    
    system_mappings = Graph()
    system_mappings.parse(system_mappings_file, format="ttl")
    
    
    #We calculate precision and recall via true positives, false positives and false negatives
    #https://en.wikipedia.org/wiki/Precision_and_recall        
    tp=0
    fp=0
    fn=0
    
    for t in system_mappings:
        if t in ref_mappings:
            tp+=1
        else:
            fp+=1

    
    for t in ref_mappings:
        if not t in system_mappings:
            fn+=1
            
        
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    #f_score = (2*precision*recall)/(precision+recall)
    print("Comparing '" + system_mappings_file + "' with '" + reference_mappings_file)
    print("\tPrecision: " + str(precision))
    print("\tRecall: " + str(recall))
    #print("\tF-Score: " + str(f_score))

In [137]:
#code taken from lab 8

reference_mappings="reference-mappings-pizza.ttl"
system_mappings="pizza_cw_equivalences.ttl"
  
compareWithReference(reference_mappings, system_mappings)

Comparing 'pizza_cw_equivalences.ttl' with 'reference-mappings-pizza.ttl
	Precision: 0.9
	Recall: 0.8181818181818182


### Subtask OA.3

In [138]:
#reasoning with everything

g = Graph()
g.parse('generated_rdf.ttl', format = 'ttl')
g.parse('pizza-restaurants-ontology.ttl', format = 'ttl')
g.parse('pizza.ttl', format = 'ttl')
g.parse('pizza_cw_equivalences.ttl', format = 'ttl')

<Graph identifier=Nd192fdd8e0404bc9919847139e1ac2b2 (<class 'rdflib.graph.Graph'>)>

In [139]:
g.serialize('all_ontologies.ttl', format = 'ttl')

In [140]:
print("triples before reasoning: {}".format(len(g)))

triples before reasoning: 5520


In [141]:
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(g)

In [142]:
print("triples after reasoning: {}".format(len(g)))

triples after reasoning: 39489


In [143]:
g.serialize('all_ontologies_after_reasoning.ttl', format = 'ttl')

### Subtask OA.4

we are counting the number of vegetarian pizzas that do not have tomoto as an ingredient using pizza:

In [144]:
query_sparql_6 = """
SELECT (COUNT(DISTINCT ?vegpizza) AS ?VegPizzasNoTomato)
WHERE {
  ?vegpizza rdf:type pizza:VegetarianPizza .       
  FILTER NOT EXISTS {                              
    ?vegpizza pizza:hasTopping ?topping .
    ?topping rdf:type pizza:TomatoTopping .
  }
}
"""


query_sparql_6_results = g.query(query_sparql_6)

print("Query Results:")
for row in query_sparql_6_results:
    print(row)

Query Results:
(rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


In [145]:
g.query(query_sparql_6).serialize(destination='query_sparql_6_results.csv', format='csv')